In [0]:
os.mkdir("data")
os.mkdir("data/gt")
os.mkdir("data/sat")

In [1]:
!pip install tifffile
!pip install libtiff

## Requirements

In [2]:
# u-net model with up-convolution or up-sampling and weighted binary-crossentropy as loss func
import os
import numpy as np
import tifffile as tiff
import math
# import libtiff
import scipy.misc
from skimage import io
from PIL import Image
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, Conv2DTranspose, BatchNormalization, Dropout
from keras.optimizers import Adam
from keras.utils import plot_model
from keras import backend as K
from keras.callbacks import CSVLogger
from keras.callbacks import TensorBoard
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.models import model_from_json

Using TensorFlow backend.


## Parameters

In [0]:
N_BANDS = 4
N_CLASSES = 8  # Roads, Trees, Bare Soil, Rails, Buildings, Grass, Water, Pools
CLASS_WEIGHTS = [0.2, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
N_EPOCHS = 150
UPCONV = True
PATCH_SZ = 112   # should divide by 16
BATCH_SIZE = 150
TRAIN_SZ = 4000  # train size
VAL_SZ = 1000    # validation size
loc = 'data/'

## Utility Functions

In [0]:
def normalize(img):
    min = img.min()
    max = img.max()
    x = 2.0 * (img - min) / (max - min) - 1.0
    return x

In [0]:
def rowslice(image, gtimage):
    n = math.floor(image.shape[1]/(PATCH_SZ//2))
    w = image.shape[1]
    rarr = range(n+1)
    X_row = []
    Y_row = []
    for i in rarr[:n-1]:
        X_row += [image[:, (PATCH_SZ//2)*i:(PATCH_SZ//2)*(i+2), :]]
        Y_row += [gtimage[:, (PATCH_SZ//2)*i:(PATCH_SZ//2)*(i+2), :]]
    if image.shape[1] % (PATCH_SZ//2) != 0:
        X_row += [image[:, w-PATCH_SZ:, :]]
        Y_row += [gtimage[:, w-PATCH_SZ:, :]]
    return [X_row, Y_row]

In [0]:
def slice(image, gtimage):
    m = math.floor(image.shape[0]/(PATCH_SZ//2))
    h = image.shape[0]
    rarr = range(m+1)
    X = []
    Y = []
    for i in rarr[:m-1]:
        [X_row, Y_row] = rowslice(image[(PATCH_SZ//2)*i:(PATCH_SZ//2)*(i+2), :, :], gtimage[(PATCH_SZ//2)*i:(PATCH_SZ//2)*(i+2), :, :])
        for X_ in X_row:
            X += [X_]
        for Y_ in Y_row:
            Y += [Y_]
    if h % (PATCH_SZ//2) != 0:
        [X_row, Y_row] = rowslice(image[h-PATCH_SZ:, :, :], gtimage[h-PATCH_SZ:, :, :])
        for X_ in X_row:
            X += [X_]
        for Y_ in Y_row:
            Y += [Y_]
    return [X, Y]

## Model

Model Image

In [0]:
def unet_model(n_classes=5, im_sz=160, n_channels=8, n_filters_start=32, growth_factor=2, upconv=True,
               class_weights=[0.2, 0.3, 0.1, 0.1, 0.3]):
    droprate=0.25
    n_filters = n_filters_start
    inputs = Input((im_sz, im_sz, n_channels))
    #inputs = BatchNormalization()(inputs)
    conv1 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(inputs)
    conv1 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    #pool1 = Dropout(droprate)(pool1)

    n_filters *= growth_factor
    pool1 = BatchNormalization()(pool1)
    conv2 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    pool2 = Dropout(droprate)(pool2)

    n_filters *= growth_factor
    pool2 = BatchNormalization()(pool2)
    conv3 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    pool3 = Dropout(droprate)(pool3)

    n_filters *= growth_factor
    pool3 = BatchNormalization()(pool3)
    conv4_0 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(pool3)
    conv4_0 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv4_0)
    pool4_1 = MaxPooling2D(pool_size=(2, 2))(conv4_0)
    pool4_1 = Dropout(droprate)(pool4_1)

    n_filters *= growth_factor
    pool4_1 = BatchNormalization()(pool4_1)
    conv4_1 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(pool4_1)
    conv4_1 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv4_1)
    pool4_2 = MaxPooling2D(pool_size=(2, 2))(conv4_1)
    pool4_2 = Dropout(droprate)(pool4_2)

    n_filters *= growth_factor
    conv5 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(pool4_2)
    conv5 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv5)

    n_filters //= growth_factor
    if upconv:
        up6_1 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv5), conv4_1])
    else:
        up6_1 = concatenate([UpSampling2D(size=(2, 2))(conv5), conv4_1])
    up6_1 = BatchNormalization()(up6_1)
    conv6_1 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(up6_1)
    conv6_1 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv6_1)
    conv6_1 = Dropout(droprate)(conv6_1)

    n_filters //= growth_factor
    if upconv:
        up6_2 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv6_1), conv4_0])
    else:
        up6_2 = concatenate([UpSampling2D(size=(2, 2))(conv6_1), conv4_0])
    up6_2 = BatchNormalization()(up6_2)
    conv6_2 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(up6_2)
    conv6_2 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv6_2)
    conv6_2 = Dropout(droprate)(conv6_2)

    n_filters //= growth_factor
    if upconv:
        up7 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv6_2), conv3])
    else:
        up7 = concatenate([UpSampling2D(size=(2, 2))(conv6_2), conv3])
    up7 = BatchNormalization()(up7)
    conv7 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(up7)
    conv7 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv7)
    conv7 = Dropout(droprate)(conv7)

    n_filters //= growth_factor
    if upconv:
        up8 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv7), conv2])
    else:
        up8 = concatenate([UpSampling2D(size=(2, 2))(conv7), conv2])
    up8 = BatchNormalization()(up8)
    conv8 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(up8)
    conv8 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv8)
    conv8 = Dropout(droprate)(conv8)

    n_filters //= growth_factor
    if upconv:
        up9 = concatenate([Conv2DTranspose(n_filters, (2, 2), strides=(2, 2), padding='same')(conv8), conv1])
    else:
        up9 = concatenate([UpSampling2D(size=(2, 2))(conv8), conv1])
    conv9 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(up9)
    conv9 = Conv2D(n_filters, (3, 3), activation='relu', padding='same')(conv9)

    conv10 = Conv2D(n_classes, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    def weighted_binary_crossentropy(y_true, y_pred):
        class_loglosses = K.mean(K.binary_crossentropy(y_true, y_pred), axis=[0, 1, 2])
        return K.sum(class_loglosses * K.constant(class_weights))

    model.compile(optimizer=Adam(), loss=weighted_binary_crossentropy)
    return model

In [0]:
def get_model():
    return unet_model(N_CLASSES, PATCH_SZ, n_channels=N_BANDS, upconv=UPCONV, class_weights=CLASS_WEIGHTS)

## Preparation for Training

In [0]:
weights_path = 'weights'
if not os.path.exists(weights_path):
    os.makedirs(weights_path)
weights_path += '/weights.hdf5'

# all availiable ids: from "1" to "14"
trainIds = [str(i) for i in range(1, 15)]

In [0]:
X = []
Y = []
for i in range(1, 15):
    addri = loc+'sat/'+str(i)+'.tif'
    addrgti = loc+'gt/'+str(i)+'.tif'
    image = np.array(io.imread(addri))
    gtimage = io.imread(addrgti)
    gtimage = np.array(gtimage)
    [X1, Y1] = slice(image, gtimage)
#     print(image.shape,gtimage.shape)
    X += X1
    Y += Y1

In [37]:
print(len(X))
part = 75
x_train = [X[:len(X)*part//100]]
y_train = [Y[:len(Y)*part//100]]
x_val = [X[len(X)*part//100:]]
y_val = [Y[len(Y)*part//100:]]
print(np.array(x_train).shape, np.array(y_val).shape)

5535
(1, 4151, 112, 112, 4) (1, 1384, 112, 112, 3)


## Training

In [0]:
def train_net():
    print("start train net")
#     x_train, y_train = get_patches(
#         X_DICT_TRAIN, Y_DICT_TRAIN, n_patches=TRAIN_SZ, sz=PATCH_SZ)
#     x_val, y_val = get_patches(
#         X_DICT_VALIDATION, Y_DICT_VALIDATION, n_patches=VAL_SZ, sz=PATCH_SZ)
    model = get_model()
    if os.path.isfile(weights_path):
        model.load_weights(weights_path)
    #model_checkpoint = ModelCheckpoint(weights_path, monitor='val_loss', save_weights_only=True, save_best_only=True)
    #early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, mode='auto')
    #reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5, min_lr=0.00001)
    model_checkpoint = ModelCheckpoint(
        weights_path, monitor='val_loss', save_best_only=True)
    csv_logger = CSVLogger('log_unet.csv', append=True, separator=';')
    tensorboard = TensorBoard(
        log_dir='./tensorboard_unet/', write_graph=True, write_images=True)
    model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs=N_EPOCHS,
              verbose=2, shuffle=True,
              callbacks=[model_checkpoint, csv_logger, tensorboard],
              validation_data=(x_val, y_val))
    return model

In [39]:
# loadImages()
model = train_net()

start train net


ValueError: ignored

## Evaluation

In [0]:
# evaluate the model
scores = model.evaluate(x_train, y_train, verbose=0)
scores_val = model.evaluate(x_val, y_val, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

## Saving the model

In [0]:
# serialize model to JSON
model_json = model.to_json()
with open(os.path.join("weights", "model.json"), "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights(os.path.join("weights", "weights.hdf5"))
print("Saved model to disk")

## Prediction

In [0]:
def predict(x, model, patch_sz=160, n_classes=5):
    img_height = x.shape[0]
    img_width = x.shape[1]
    n_channels = x.shape[2]
    # make extended img so that it contains integer number of patches
    npatches_vertical = math.ceil(img_height / patch_sz)
    npatches_horizontal = math.ceil(img_width / patch_sz)
    extended_height = patch_sz * npatches_vertical
    extended_width = patch_sz * npatches_horizontal
    ext_x = np.zeros(shape=(extended_height, extended_width,
                            n_channels), dtype=np.float32)
    # fill extended image with mirrors:
    ext_x[:img_height, :img_width, :] = x
    for i in range(img_height, extended_height):
        ext_x[i, :, :] = ext_x[2 * img_height - i - 1, :, :]
    for j in range(img_width, extended_width):
        ext_x[:, j, :] = ext_x[:, 2 * img_width - j - 1, :]

    # now we assemble all patches in one array
    patches_list = []
    for i in range(0, npatches_vertical):
        for j in range(0, npatches_horizontal):
            x0, x1 = i * patch_sz, (i + 1) * patch_sz
            y0, y1 = j * patch_sz, (j + 1) * patch_sz
            patches_list.append(ext_x[x0:x1, y0:y1, :])
    # model.predict() needs numpy array rather than a list
    patches_array = np.asarray(patches_list)
    # predictions:
    patches_predict = model.predict(patches_array, batch_size=4)
    prediction = np.zeros(
        shape=(extended_height, extended_width, n_classes), dtype=np.float32)
    for k in range(patches_predict.shape[0]):
        i = k // npatches_horizontal
        j = k % npatches_vertical
        x0, x1 = i * patch_sz, (i + 1) * patch_sz
        y0, y1 = j * patch_sz, (j + 1) * patch_sz
        prediction[x0:x1, y0:y1, :] = patches_predict[k, :, :, :]
    return prediction[:img_height, :img_width, :]

In [0]:
def picture_from_mask(mask, threshold=0):
    colors = {
        0: [150, 150, 150],  # Roads
        1: [223, 194, 125],  # Trees
        2: [27, 120, 55],    # Bare Soil
        3: [166, 219, 160],  # Building
        4: [116, 173, 209]   # Grass
        5: [169,169,169]     # Water
        6: [0,191,255]       # Pools
    }
    z_order = {
        1: 3,
        2: 4,
        3: 5,
        4: 6,
        5: 7,
        6: 8,
        7: 1,
        8: 2
    }
    pict = 255*np.ones(shape=(3, mask.shape[1], mask.shape[2]), dtype=np.uint8)
    for i in range(1, 6):
        cl = z_order[i]
        for ch in range(3):
            pict[ch, :, :][mask[cl, :, :] > threshold] = colors[cl][ch]
    return pict



In [0]:
# load json and create model
jsonFile = open(os.path.join("weights", "model.json"), 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
# load weights into new model
model.load_weights("model.h5")
print("Loaded model from disk")

In [0]:
test_id = '4'
img = normalize(tiff.imread('data/sat/{}.tif'.format(test_id)).transpose([1, 2, 0]))   # make channels last

for i in range(7):
    if i == 0:  # reverse first dimension
        mymat = predict(img[::-1, :, :], model, patch_sz=PATCH_SZ,
                        n_classes=N_CLASSES).transpose([2, 0, 1])
        #print(mymat[0][0][0], mymat[3][12][13])
        print("Case 1", img.shape, mymat.shape)
    elif i == 1:    # reverse second dimension
        temp = predict(img[:, ::-1, :], model, patch_sz=PATCH_SZ,
                       n_classes=N_CLASSES).transpose([2, 0, 1])
        #print(temp[0][0][0], temp[3][12][13])
        print("Case 2", temp.shape, mymat.shape)
        mymat = np.mean(np.array([temp[:, ::-1, :], mymat]), axis=0)
    elif i == 2:    # transpose(interchange) first and second dimensions
        temp = predict(img.transpose(
            [1, 0, 2]), model, patch_sz=PATCH_SZ, n_classes=N_CLASSES).transpose([2, 0, 1])
        #print(temp[0][0][0], temp[3][12][13])
        print("Case 3", temp.shape, mymat.shape)
        mymat = np.mean(np.array([temp.transpose(0, 2, 1), mymat]), axis=0)
    elif i == 3:
        temp = predict(np.rot90(img, 1), model,
                       patch_sz=PATCH_SZ, n_classes=N_CLASSES)
        #print(temp.transpose([2,0,1])[0][0][0], temp.transpose([2,0,1])[3][12][13])
        print("Case 4", temp.shape, mymat.shape)
        mymat = np.mean(
            np.array([np.rot90(temp, -1).transpose([2, 0, 1]), mymat]), axis=0)
    elif i == 4:
        temp = predict(np.rot90(img, 2), model,
                       patch_sz=PATCH_SZ, n_classes=N_CLASSES)
        #print(temp.transpose([2,0,1])[0][0][0], temp.transpose([2,0,1])[3][12][13])
        print("Case 5", temp.shape, mymat.shape)
        mymat = np.mean(
            np.array([np.rot90(temp, -2).transpose([2, 0, 1]), mymat]), axis=0)
    elif i == 5:
        temp = predict(np.rot90(img, 3), model,
                       patch_sz=PATCH_SZ, n_classes=N_CLASSES)
        #print(temp.transpose([2,0,1])[0][0][0], temp.transpose([2,0,1])[3][12][13])
        print("Case 6", temp.shape, mymat.shape)
        mymat = np.mean(
            np.array([np.rot90(temp, -3).transpose(2, 0, 1), mymat]), axis=0)
    else:
        temp = predict(img, model, patch_sz=PATCH_SZ,
                       n_classes=N_CLASSES).transpose([2, 0, 1])
        #print(temp[0][0][0], temp[3][12][13])
        print("Case 7", temp.shape, mymat.shape)
        mymat = np.mean(np.array([temp, mymat]), axis=0)

#print(mymat[0][0][0], mymat[3][12][13])
map = picture_from_mask(mymat, 0.5)
# mask = predict(img, model, patch_sz=PATCH_SZ, n_classes=N_CLASSES).transpose([2,0,1])  # make channels first
#map = picture_from_mask(mask, 0.5)

#tiff.imsave('result.tif', (255*mask).astype('uint8'))
tiff.imsave('result.tif', (255*mymat).astype('uint8'))
tiff.imsave('map.tif', map)